In [1]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"]="python"
import numpy as np
import matlab
import matlab.engine
import paddle
import paddle_quantum
from paddle_quantum.ansatz import Circuit
from paddle_quantum.state import State
from paddle_quantum.qinfo import partial_trace, random_channel
from scipy import optimize
from typing import List

paddle_quantum.set_backend('density_matrix')
paddle_quantum.set_dtype('complex128')


d:\Anaconda\envs\pq\lib\site-packages\openfermion\hamiltonians\hartree_fock.py:11: DeprecationWarning: Please use `OptimizeResult` from the `scipy.optimize` namespace, the `scipy.optimize.optimize` namespace is deprecated.
  from scipy.optimize.optimize import OptimizeResult
d:\Anaconda\envs\pq\lib\site-packages\paddle\tensor\creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
d:\Anaconda\envs\pq\lib\site-packages\paddle\tensor\creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://num

In [2]:
NUM_QUBIT = 2
NUM_LAYER = 10
SEED = np.random.randint(0, 1000000)

In [3]:
eng = matlab.engine.start_matlab()

def __count_params(total_dim) -> int:
    r"""Count the number of parameters of the model
    """
    num_param = total_dim
    for i in range(1, total_dim):
        for _ in range(i - 1):
            num_param += 2
    return num_param


def init_param() -> paddle.Tensor:
    r"""Initialize the parameters of the model
    """
    return paddle.uniform([NUM_LAYER, NUM_QUBIT, 1], dtype='float64', min=0, max=2 * np.pi, seed=SEED) # random initial theta for ry/u3


# circuit for calculating upper bound
def QNN(num_qubit, num_layer, theta):
    cir = Circuit(num_qubit)
    theta = paddle.reshape(paddle.to_tensor(theta), [num_layer, num_qubit, 1])
    # append ry/u3 gates
    for repeat in range(num_layer):
        for i in range(num_qubit):
            cir.ry(qubits_idx=i, param=theta[repeat][i][0])
        for j in range(num_qubit - 1):
            cir.cnot([j, j + 1])
        cir.cnot([num_qubit-1, 0])
    return cir


def record_best(cur_theta, optimizer):
    cur_cost = compute_loss(cur_theta, optimizer.num_layer)
    if  cur_cost < optimizer.min_cost:
        optimizer.min_cost = cur_cost
        optimizer.min_theta = cur_theta
        print(optimizer.min_cost)
    return cur_cost
    

# compute loss function via SDP
def compute_loss(theta, depth, r_channel, qubit_B):
    cir = QNN(NUM_QUBIT, depth, theta)
    cir.choi_channel(r_channel, qubits_idx=qubit_B) # 2-qubit
    state = cir()
    
    # use MATLAB to compute loss of the extension state
    qbound = eng.Q_bound_1ext(matlab.double(state.numpy().tolist(), is_complex=True))
    GLOBALVAL.append(qbound)
    GLOBALSTATE.append(state.numpy())
    return qbound


# mixed unitary channel
def rand_mixed_unitary(p: List[float]):
    JU = eng.RandMixedU(matlab.double(p, is_complex=False))
    return np.array(JU)

# amplitude damping channel
def ad_depo_qubit(pA: float, pD: float):
    JU = eng.AD_Depo_qubit(matlab.double(pA, is_complex=False),
                           matlab.double(pD, is_complex=False))
    return np.array(JU)

# random pauli channel
def rand_pauli_channel(p: List[float]):
    JPa = eng.PauliChan(matlab.double(p, is_complex=False))
    return np.array(JPa)


## Random channel bound sampling

In [4]:
from paddle_quantum.state import bell_state

In [34]:
SAMP = 10
pA_list = np.arange(0.0, 0.8, 0.01)
BOUNDRECORD = []
learning_rate = 0.07
iterations = 1
bell = bell_state(2)

d:\Anaconda\envs\pq\lib\site-packages\paddle\fluid\framework.py:1104: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:


## AD channel

In [ ]:
SEED = np.random.randint(0, 1000000)
init_theta = paddle.uniform([NUM_LAYER, NUM_QUBIT, 1], dtype='float64', min=0, max=2 * np.pi, seed=SEED) # random initial theta for ry/u3
for pA in pA_list:
    GLOBALVAL = []
    GLOBALSTATE = []
    r_channel = paddle.to_tensor(ad_depo_qubit(1-pA, 0.0))
    bellbound = eng.Q_bound_1ext(matlab.double((r_channel).tolist(), is_complex=True))
    LRb = eng.LRbound(matlab.double((r_channel).tolist(), is_complex=True))
    BOUNDRECORD.append([-2*np.log2(min([0])), -2*np.log2(bellbound), LRb])
    np.save(f'bound_record_1102_AD_{SEED}.npy', BOUNDRECORD)

In [33]:
import scipy

scipy.io.savemat('pA_AD_Q.mat', {'data': pA_list})
scipy.io.savemat('data_AD_Q.mat', {'data': data})

## Random Channel sampling

In [6]:
SEED = np.random.randint(0, 1000000)
init_theta = paddle.uniform([NUM_LAYER, NUM_QUBIT, 1], dtype='float64', min=0, max=2 * np.pi, seed=73) # random initial theta for ry/u3
for _ in range(SAMP):
    GLOBALVAL = []
    GLOBALSTATE = []
    # r_channel = random_channel(1, target='Choi')*(1/2 + 0j)
    r_channel = paddle.to_tensor(rand_mixed_unitary([0.4, 0.4, 0.1, 0.1])) * (1/(2**2) + 0j)
    
    init_theta = init_param()
    opt_res = optimize.minimize(compute_loss, init_theta, args=(NUM_LAYER, (2**2 + 0j)*r_channel, [2,3]), method='Powell', options={'maxiter': iterations})
    bellbound = eng.Q_bound_1ext(matlab.double((r_channel).tolist(), is_complex=True))
    LRb = eng.LRbound(matlab.double((r_channel).tolist(), is_complex=True))
    BOUNDRECORD.append([-2*np.log2(min(GLOBALVAL)), -2*np.log2(bellbound), LRb])
    np.save(f'bound_record_1102_{SEED}.npy', BOUNDRECORD)
    if _ % 10 == 0:
        print('Sample: ', _)


C:\Users\johni\AppData\Local\Temp\ipykernel_3596\3669841296.py:9: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  opt_res = optimize.minimize(compute_loss, init_theta, args=(NUM_LAYER, (2**2 + 0j)*r_channel, [2,3]), method='Powell', options={'maxiter': iterations})


Sample:  0
